In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from torchinfo import summary

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt

import re
from konlpy.tag import Mecab

from gensim.models import word2vec

import sys
sys.path.append('..')

from python_scripts import data_setup, engine


In [2]:
idx_to_class = {
  0: 'Origin(출신차별)',
  1: 'Physical(외모차별) 외모(신체, 얼굴) 및 장애인 차별 발언을 포함합니다.',
  2: 'Politics(정치성향차별)',
  3: 'Profanity(혐오욕설) 욕설,저주,혐오 단어, 비속어 및 기타 혐오 발언을 포함합니다.',
  4: 'Age(연령차별)',
  5: 'Gender(성차별) 성별 또는 성적 취향에 대한 차별 발언을 포함합니다.',
  6: 'Not Hate Speech(해당사항없음)',
}

max_length = 200

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [4]:
train_data = pd.read_csv('../data/Discrimination/train.csv')
train_data

,ID,text,label
0,0,유소영비호감 성형아줌마,1
1,1,나오지마라 썅,3
2,2,식상하고 지긋지긋했는데 잘 끝나네 오예 소리벗고 빤스질러~~!!!,6
3,3,성희롱 당할 얼굴이 아닌데?ㅋㅋㅋ,5
4,4,"""끝까지 해보자~쪽파리 원숭이 자한 쓰레기당""",0
...,...,...,...
65858,65858,ㅋ ㅋ 쇼~~~ 도 적당히,6
65859,65859,"""이젠 전라도 종것들 음식 불매다. 태양광 중금속 환경오염물로 만든 식품 사절이다""",0
65860,65860,조센징들은 참 피곤하게 산다,0
65861,65861,"""문빠 다모아서 빨갱이한테보내고 행복하게살라고""",2


In [5]:
max([len(s) for s in train_data['text']])

306

In [10]:
def preprocess_korean_text(text):
    # Remove URLs and mentions
    text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "", text)
    text = re.sub(r"@(\w+)", "", text)

    # Tokenize text using Mecab
    mecab = Mecab()
    tokens = mecab.morphs(text)

    # Remove stop words (optional)
    stop_words = ["은", "는", "이", "가", "을", "를", "에", "의", "로", "으로", "에서"]
    tokens = [t for t in tokens if t not in stop_words]

    # Remove punctuation and non-Korean characters
    tokens = [re.sub(r"[^\u3131-\u3163\uac00-\ud7a3]+", "", t) for t in tokens]
    tokens = [t for t in tokens if t]

    return tokens

In [11]:
preprocess_korean_text('나는 지금 뭐하고 있느냐?')

['나', '지금', '뭐', '하', '고', '있', '느냐']

In [12]:
# Getting Word2Vec embedding pre-trained model

w2v_pretrained_model = word2vec.Word2Vec.load('../data/Discrimination/word2vec')
w2v_pretrained_model.wv.add_vector('<unk>', [0.0] * 100)

/Users/tglim/miniforge3/envs/tensorflow/lib/python3.8/site-packages/gensim/models/keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(


358043

In [13]:
# Getting GloVe embedding pre-trained model

def load_glove_model(file):
    print("Loading Glove Weight")
    glove_vector = {}
    with open(file,'r') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_vector[word] = embedding

    # 없는 단어 추가
    not_exists = pd.read_csv('../data/Discrimination/not_exist_list.csv')
    for i in range(len(not_exists)):
        glove_vector[not_exists.loc[i, 'word']] = glove_vector[not_exists.loc[i, 'substitute']]

    class Word_vector():
        def __init__(self, key_to_vector) -> None:
            self.key_to_vector = key_to_vector

            self.index_to_key = []
            self.key_to_index = {}
            for key in self.key_to_vector.keys():
                self.index_to_key.append(key)
                self.key_to_index[key] = len(self.index_to_key) - 1

            self.vectors = []
            for i in range(len(self.index_to_key)):
                self.vectors.append(self.key_to_vector[self.index_to_key[i]])
            self.vectors = np.array(self.vectors, dtype='float32')

            self.vector_size = len(self.vectors[0])

        def __contains__(self, key):
            return key in self.key_to_vector

        def __getitem__(self, key):
            return self.key_to_vector[key]

        def __len__(self):
            return len(self.index_to_key)

    class Glove_model():
        def __init__(self, vector) -> None:
            self.wv = Word_vector(vector)

        def __len__(self):
            return len(self.wv)

    glove_model = Glove_model(glove_vector)

    print(f"{len(glove_model)} words loaded!")
    return glove_model

glove_pretrained_model = load_glove_model('../data/Discrimination/glove.txt')
len(glove_pretrained_model.wv.index_to_key)

Loading Glove Weight
358085 words loaded!


358085

In [14]:
# pre-trained 임베딩 벡터에 등록되지 않은 단어 중 주요 단어를 골라 수작업 처리
# all = 0
# not_exists = {}
# not_exists_labels = {}

# for i in tqdm_notebook(range(len(train_data))):
#     sentence, label = train_data.loc[i, 'text'], train_data.loc[i, 'label']
#     for word in preprocess_korean_text(sentence):
#         if word not in glove_pretrained_model.wv:
#             if word in not_exists:
#                 not_exists[word] += 1
#             else:
#                 not_exists[word] = 1

#             if word not in not_exists_labels:
#                 not_exists_labels[word] = {n: 0 for n in range(7)}
#             not_exists_labels[word][label] += 1
#         all += 1

# not_exists = sorted(not_exists.items(), key=lambda x: x[1], reverse=True)
# not_labels = []
# for word, n in not_exists:
#     not_labels.append(sorted(not_exists_labels[word].items(), key=lambda x: x[1], reverse=True))
# print(all, len(not_exists))
# print(not_labels)

In [15]:
# not_exist_list = pd.DataFrame({'word': not_exists, 'label': not_labels})
# not_exist_list.to_csv('../data/Discrimination/not_exist_list.csv', index=False)

In [16]:
# # pre-trained embedding model 이 없는 경우.
# # 이에 맞추어 Dataset class 와 model 에 대해서도 수정이 필요하게 됨

# word_index_to_key = []
# word_key_to_index = {}

# for i in tqdm_notebook(range(len(train_data)), 'Making word maps'):
#     text = train_data.iloc[i]['text']
#     tokens = preprocess_korean_text(text)

#     for token in tokens:
#         if token not in word_key_to_index:
#             word_key_to_index[token] = len(word_index_to_key)
#             word_index_to_key.append(token)

# word_key_to_index['<unk>'] = len(word_index_to_key)
# word_index_to_key.append('<unk>')

# len(word_index_to_key)

In [17]:
class KoreanTextDataset(Dataset):
    def __init__(self, data, embed_model, preprocess_korean_text, max_length=100):
        self.data = data
        self.max_length = max_length
        self.preprocess_korean_text = preprocess_korean_text
        self.model = embed_model
        self.idx_to_class = sorted(data['label'].unique())
        self.class_to_idx = {}
        for i in range(len(self.idx_to_class)):
            self.class_to_idx[self.idx_to_class[i]] = i
        self.class_names = self.idx_to_class

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.loc[index, "text"]
        label = self.data.loc[index, "label"]

        # Preprocess text using the preprocess_korean_text() function
        tokens = self.preprocess_korean_text(text)
        # Truncate or pad tokens to a fixed length
        if len(tokens) > self.max_length:
            tokens = tokens[:self.max_length]
        else:
            tokens += [""] * (self.max_length - len(tokens))

        # Convert tokens to indices using the pre-trained GloVe or Word2Vec embeddings
        indices = []
        for token in tokens:
            if token in self.model.wv:
                indices.append(self.model.wv.key_to_index[token])
            else:
                indices.append(self.model.wv.key_to_index['<unk>'])  # use the index of the <unk> token for out-of-vocabulary words

        return torch.tensor(indices), torch.tensor(label)

In [18]:
embed_model = glove_pretrained_model

train_dataset = KoreanTextDataset(
    data=train_data,
    embed_model=embed_model,
    preprocess_korean_text=preprocess_korean_text,
    max_length=max_length
)

train_dataset_sub, val_dataset_sub = data_setup.split_dataset(
    dataset=train_dataset,
    split_size=0.999,
    seed=42
)

[INFO] Splitting dataset of length 65863 into splits of size: 65797 and 66


In [19]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(0)

        repeated_hidden = hidden.unsqueeze(0).repeat(max_len, 1, 1)

        energy = torch.tanh(self.attn(torch.cat((repeated_hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)
        return F.softmax(attention, dim=0).unsqueeze(2)

In [ ]:
# version 1
class RNN_LSTM_attention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, LSTM_layers, dropout):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        for param in self.embedding.parameters():
            param.requires_grad = False

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=LSTM_layers, bidirectional=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.attention = Attention(hidden_dim * 2)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, text):
        # text = [batch size, seq len]

        embedded = self.embedding(text)
        embedded = self.dropout(embedded)
        # embedded = [batch size, seq len, emb dim]: [64, 200, 100]
        # print('embedded: ', embedded.shape)

        lstm_outputs, (hidden, _) = self.lstm(embedded.permute(1, 0, 2))
        # output = [batch size, seq len, hid dim * num directions]: [200, 64, 1024]
        # hidden/cell = [num layers * num directions, batch size, hid dim]: [6, 64, 512]
        # print('outputs, hidden: ', pre_lstm_outputs.shape, hidden.shape)

        h = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        # [64, 1024]
        # print('h: ', h.shape)

        attention_weights = self.attention(h, lstm_outputs)
        # # attention_weights = [batch size, seq len, 1]: [200, 64, 1]
        # print('attention_weights: ', attention_weights.shape)

        context_vector = torch.bmm(lstm_outputs.permute(1, 2, 0), attention_weights.permute(1, 0, 2)).squeeze(2)
        # # context_vector = [batch size, hid dim * num directions]: [64, 1024]
        # print('context_vector: ', context_vector.shape)

        out = self.fc(self.dropout(context_vector.squeeze(0)))
        # out = [batch size, output dim]: [64, 7]
        # print('out: ', out.shape)

        return out

In [20]:
# version 2
class RNN_LSTM_attention(nn.Module):
    def __init__(self, embedding_model, hidden_dim, output_dim, pre_LSTM_layers, post_LSTM_layers, dropout):
        super().__init__()

        vocab_size = len(embedding_model.wv.index_to_key)
        embedding_dim = embedding_model.wv.vector_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim, _weight=torch.tensor(embedding_model.wv.vectors))
        for param in self.embedding.parameters():
            param.requires_grad = False

        self.pre_lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=pre_LSTM_layers, bidirectional=True, dropout=dropout)
        self.post_lstm = nn.LSTM(hidden_dim * 2, hidden_dim, num_layers=post_LSTM_layers, bidirectional=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.attention = Attention(hidden_dim * 2)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, text):
        # text = [batch size, seq len]

        embedded = self.embedding(text)
        embedded = self.dropout(embedded)
        # embedded = [batch size, seq len, emb dim]: [64, 200, 100]
        # print('embedded: ', embedded.shape)

        pre_lstm_outputs, (hidden, cell) = self.pre_lstm(text.permute(1, 0, 2))
        # output = [batch size, seq len, hid dim * num directions]: [200, 64, 1024]
        # hidden/cell = [num layers * num directions, batch size, hid dim]: [6, 64, 512]
        # print('outputs, hidden: ', pre_lstm_outputs.shape, hidden.shape)

        h = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        # [64, 1024]
        # print('h: ', h.shape)

        attention_weights = self.attention(h, pre_lstm_outputs)
        # # attention_weights = [batch size, seq len, 1]: [200, 64, 1]
        # print('attention_weights: ', attention_weights.shape)

        context_vector = torch.bmm(pre_lstm_outputs.permute(1, 2, 0), attention_weights.permute(1, 0, 2)).squeeze(2)
        # # context_vector = [batch size, hid dim * num directions]: [64, 1024]
        # print('context_vector: ', context_vector.shape)

        _, (hidden, _) = self.post_lstm(context_vector.unsqueeze(0), (hidden, cell))
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        # hidden = [batch size, hid dim * num directions]: [64, 1024]
        # print('hidden: ', hidden.shape)

        out = self.fc(self.dropout(hidden.squeeze(0)))
        # out = [batch size, output dim]: [64, 7]
        # print('out: ', out.shape)

        return out


In [ ]:
# test
class RNN_LSTM_attention(nn.Module):
    def __init__(self, embedding_model, hidden_dim, output_dim, pre_LSTM_layers, post_LSTM_layers, dropout):
        super().__init__()

        vocab_size = len(embedding_model.wv.index_to_key)
        embedding_dim = embedding_model.wv.vector_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim, _weight=torch.tensor(embedding_model.wv.vectors))
        for param in self.embedding.parameters():
            param.requires_grad = False

        self.pre_lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=pre_LSTM_layers, bidirectional=True, dropout=dropout)
        self.post_lstm = nn.LSTM(hidden_dim * 2, hidden_dim, num_layers=post_LSTM_layers, bidirectional=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.attention = Attention(hidden_dim * 2)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, text):
        # text = [batch size, seq len]

        # embedded = self.embedding(text)
        # embedded = self.dropout(embedded)
        # embedded = [batch size, seq len, emb dim]: [64, 200, 100]
        # print('embedded: ', embedded.shape)

        pre_lstm_outputs, (hidden, cell) = self.pre_lstm(text.permute(1, 0, 2))
        # output = [batch size, seq len, hid dim * num directions]: [200, 64, 1024]
        # hidden/cell = [num layers * num directions, batch size, hid dim]: [6, 64, 512]
        # print('outputs, hidden: ', pre_lstm_outputs.shape, hidden.shape)
        return pre_lstm_outputs, hidden # test for understanding of output and hidden

        h = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        # [64, 1024]
        # print('h: ', h.shape)

        attention_weights = self.attention(h, pre_lstm_outputs)
        # # attention_weights = [batch size, seq len, 1]: [200, 64, 1]
        # print('attention_weights: ', attention_weights.shape)

        context_vector = torch.bmm(pre_lstm_outputs.permute(1, 2, 0), attention_weights.permute(1, 0, 2)).squeeze(2)
        # # context_vector = [batch size, hid dim * num directions]: [64, 1024]
        # print('context_vector: ', context_vector.shape)

        _, (hidden, _) = self.post_lstm(context_vector.unsqueeze(0), (hidden, cell))
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        # hidden = [batch size, hid dim * num directions]: [64, 1024]
        # print('hidden: ', hidden.shape)

        out = self.fc(self.dropout(hidden.squeeze(0)))
        # out = [batch size, output dim]: [64, 7]
        # print('out: ', out.shape)

        return out


In [21]:
model = RNN_LSTM_attention(
    embedding_model=glove_pretrained_model,
    hidden_dim=512,
    output_dim=7,
    pre_LSTM_layers=2,
    post_LSTM_layers=2,
    dropout=0.2
)
o, h = model(torch.randn((1, 200, 100)))

In [54]:
o.shape, h.shape

(torch.Size([200, 1, 1024]), torch.Size([4, 1, 512]))

In [24]:
o[-1, :].shape, h[-1, :].shape

(torch.Size([1, 1024]), torch.Size([1, 512]))

In [56]:
o[-1, :, :10], h[2, :, :10]

(tensor([[-0.0182, -0.0492,  0.0121,  0.0308, -0.0241,  0.0340, -0.0165, -0.0095,
          -0.0211,  0.0014]], grad_fn=<SliceBackward0>),
 tensor([[-0.0182, -0.0492,  0.0121,  0.0308, -0.0241,  0.0340, -0.0165, -0.0095,
          -0.0211,  0.0014]], grad_fn=<SliceBackward0>))

In [55]:
o[0, :, 512:522], h[3, :, :10]

(tensor([[ 0.0151, -0.0139,  0.0147, -0.0057, -0.0041,  0.0068, -0.0225,  0.0212,
           0.0005,  0.0166]], grad_fn=<SliceBackward0>),
 tensor([[ 0.0151, -0.0139,  0.0147, -0.0057, -0.0041,  0.0068, -0.0225,  0.0212,
           0.0005,  0.0166]], grad_fn=<SliceBackward0>))

In [24]:
learning_rate_list = [1e-3, 1e-4, 1e-5] # 각 LR 별로 10 epoch 씩 연달아 학습 진행
weight_decay_list = [1e-4]
epochs_list = [5]
batch_size_list = [64]

In [25]:
class_names, num_classes = train_dataset.class_names, len(train_dataset.class_names)
class_names, num_classes

([0, 1, 2, 3, 4, 5, 6], 7)

In [26]:
model_w2v = RNN_LSTM_attention(
    embedding_model=w2v_pretrained_model,
    hidden_dim=512,
    output_dim=num_classes,
    pre_LSTM_layers=2,
    post_LSTM_layers=2,
    dropout=0.2
)

In [27]:
model_glove = RNN_LSTM_attention(
    embedding_model=glove_pretrained_model,
    hidden_dim=512,
    output_dim=num_classes,
    pre_LSTM_layers=2,
    post_LSTM_layers=2,
    dropout=0.2
)

In [28]:
summary(model_w2v), summary(model_glove)

(=================================================================
 Layer (type:depth-idx)                   Param #
 RNN_LSTM_attention                       --
 ├─Embedding: 1-1                         (35,804,400)
 ├─LSTM: 1-2                              8,814,592
 ├─LSTM: 1-3                              12,599,296
 ├─Dropout: 1-4                           --
 ├─Attention: 1-5                         --
 │    └─Linear: 2-1                       2,098,176
 │    └─Linear: 2-2                       1,024
 ├─Linear: 1-6                            7,175
 Total params: 59,324,663
 Trainable params: 23,520,263
 Non-trainable params: 35,804,400
 =================================================================,
 Layer (type:depth-idx)                   Param #
 RNN_LSTM_attention                       --
 ├─Embedding: 1-1                         (35,808,500)
 ├─LSTM: 1-2                              8,814,592
 ├─LSTM: 1-3                              12,599,296
 ├─Dropout: 1-4            

In [29]:
model = model_glove

tuning_results = engine.HP_tune_train(
    model=model,
    model_generator=None,
    model_weights=None,
    model_name='Two_LSTM_attention_glove_whole_data_discrimination',
    train_dataset=train_dataset_sub,
    test_dataset=val_dataset_sub,
    class_names=class_names,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=False,
    device=device,
    gradient_accumulation_num=1,
    saving_max=True,
    metric_learning=False
)

Two_LSTM_attention_glove_whole_data_discrimination_LR_0.001_WD_0.0001_BS_64_GA_1:   0%|          | 0/5 [00:00<…

train:   0%|          | 0/1029 [00:00<?, ?it/s]

test:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 1.1235, Train_acc: 0.6028 | Test_loss: 0.7182, Test_acc: 0.8516
[INFO] Saving model to: ..\models\Two_LSTM_attention_glove_whole_data_discrimination_LR_0.001_WD_0.0001_BS_64_GA_1_EPOCH_0_TEST-ACC_0.8516.pth


train:   0%|          | 0/1029 [00:00<?, ?it/s]

...

test:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 3 | Train_loss: 0.5650, Train_acc: 0.7961 | Test_loss: 1.0774, Test_acc: 0.6250


train:   0%|          | 0/1029 [00:00<?, ?it/s]

test:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 4 | Train_loss: 0.5663, Train_acc: 0.7964 | Test_loss: 1.0571, Test_acc: 0.6250


In [ ]:
# test_data = pd.read_csv('../data/Discrimination/test.csv')

# tt = []
# for i in tqdm_notebook(range(len(train_data))):
#     tt.append(len(preprocess_korean_text(train_data.iloc[i]['text'])))

# plt.boxplot(tt)
# print(max(tt), min(tt), np.mean(tt), np.var(tt))
# print(np.sum(np.array(tt) > 100), '/', len(tt))

In [46]:
loaded_weight = torch.load('..\models\discrimination\Two_LSTM_attention_glove_whole_data_discrimination_LR_0.001_WD_0.0001_BS_64_GA_1_EPOCH_4_TEST-ACC_0.8828.pth')
model.load_state_dict(loaded_weight)

<All keys matched successfully>

In [47]:
test_data = pd.read_csv('../data/Discrimination/test.csv')
labels = []

model.eval()

with torch.inference_mode():
  for i in tqdm_notebook(range(len(test_data))):
    test_text = test_data.loc[i, "text"]
    test_tokens = preprocess_korean_text(test_text)
    if len(test_tokens) > max_length:
        test_tokens = test_tokens[:max_length]
    else:
        test_tokens += [""] * (max_length - len(test_tokens))

    indices = []
    for token in test_tokens:
      if token in embed_model.wv:
        indices.append(embed_model.wv.key_to_index[token])
      else:
        indices.append(embed_model.wv.key_to_index['<unk>'])  # use the index of the <unk> token for out-of-vocabulary words

    test_logits = model(torch.tensor(indices).unsqueeze(0).to(device))
    labels.append(class_names[torch.argmax(test_logits.squeeze(0).cpu())])

  0%|          | 0/13491 [00:00<?, ?it/s]

In [48]:
test_data['label'] = [idx_to_class[label] for label in labels]
test_data

,ID,text,label
0,0,"""솔직히 우리나라 청년들도 불쌍하고 아재들도 불쌍하고 노인들도 불쌍하다. 나라가 참""",Age(연령차별)
1,1,그만 보고싶네요 .늙은애들은.,Age(연령차별)
2,2,"""더러운 개신교벌레 새퀴""","Profanity(혐오욕설) 욕설,저주,혐오 단어, 비속어 및 기타 혐오 발언을 포..."
3,3,근데전태수씨 사망이유가뭔가요그어떤기사에도 나오질않네요,Not Hate Speech(해당사항없음)
4,4,"""태극기부대와 틀닭바퀴충들에게 순시리는 국모다. ㅉㅉ""",Politics(정치성향차별)
...,...,...,...
13486,13486,"""ㅅㅂ저년 한국인 아니라서 법적으로 어떻게 못하는거아님?OOO아 ㅅㅂ 느그나라로 꺼져""","Profanity(혐오욕설) 욕설,저주,혐오 단어, 비속어 및 기타 혐오 발언을 포..."
13487,13487,"""틀딱 택시기사 왔는가""",Age(연령차별)
13488,13488,"""틀딱들의 흔한 망상중 하나죠ㅋㅋ""",Age(연령차별)
13489,13489,"""빨갱이 새....끼 역시 잔인하노 능지처참은 중국놈인 니놈이고""",Politics(정치성향차별)


In [49]:
submission_data = pd.DataFrame({'ID': range(len(test_data)), 'label': labels})
submission_data.to_csv('../submissions/discrimination/submission_Two_LSTM_attention_glove_whole_data_discrimination_last.csv', index=False)
print('submission completed!')
submission_data.head()

submission completed!


,ID,label
0,0,4
1,1,4
2,2,3
3,3,6
4,4,2
